In [ ]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
from core.routing import RouteService
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [ ]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)



In [ ]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk", "data/osm/ZMG_enclosure_2km.geojson")
graph_transit = graph_loader.create_graph_transit(stops_df)

In [ ]:
route_service = RouteService(graph_walk, graph_transit, stops_df)

In [ ]:
import random as rd
from shapely.geometry import Point

#choose randomly a node from G
def random_conected_walking_nodes():

    path = []

    while path == []:
        start = rd.choice(list(graph_transit.nodes))
        destination = rd.choice(list(graph_transit.nodes))

        try:
            path = nx.shortest_path(graph_transit, source=start, target=destination, weight='weight')
            print("attempt")
        except nx.NetworkXNoPath:
            path = []
    
    #get positions of start and destination nodes
    point_start = route_service.metric_point_from_transit_node(start)
    point_destination = route_service.metric_point_from_transit_node(destination)

    #add some random noise to the points
    noise_start = Point(point_start.x + rd.uniform(-100, 100), point_start.y + rd.uniform(-100, 100))
    noise_destination = Point(point_destination.x + rd.uniform(-100, 100), point_destination.y + rd.uniform(-100, 100))

    return noise_start, noise_destination





p1, p2  = random_conected_walking_nodes()

print("distnace between points:", p1.distance(p2))


def get_walking_transit_points(point: Point):

    walk_node = route_service.nearest_walking_node(point)
    walk_node_point = route_service.metric_point_from_walking_node(walk_node)

    transit_node = route_service.nearest_transit_node(walk_node_point)

    point_transit_node = route_service.metric_point_from_transit_node(transit_node)

    walk_transit_node = route_service.nearest_walking_node(point_transit_node)

    return walk_node, walk_transit_node, transit_node



walk_start_node, walk_transit_start_node, start = get_walking_transit_points(p1)

walk_end_node, walk_transit_end_node, destination = get_walking_transit_points(p2)


walk_path_start = nx.shortest_path(graph_walk, source=walk_start_node, target=walk_transit_start_node)
walk_path_end = nx.shortest_path(graph_walk, source=walk_transit_end_node, target=walk_end_node)







print("Start transit stop:", start)
print("Destination transit stop:", destination)


In [ ]:



dijkstra_path, dijkstra_cost = route_service.dijkstra_transit(start, destination)


print("Shortest path from", start, "to", destination, ":", dijkstra_path)

print("Cost of the path:", dijkstra_cost/60)

transit_gdf = gpd.GeoDataFrame(transit_df, geometry='shape_geometry', crs='EPSG:4326')

dijkstra_path_stops = []
dijkstra_path_shapes = []
prev_shape = None
for stop, shape in dijkstra_path:
    dijkstra_path_stops.append(stop)
    dijkstra_path_shapes.append(shape)
    if prev_shape is None:
        print("Walk to stop:", stop)
    if shape == 'walking':
        print("Walk to stop:", stop)
    elif shape != prev_shape:
        print("Take Bus:", shape, "from stop:", stop, " direction to:", transit_df[transit_df['shape_id'] == shape]['trip_headsign'].iloc[0])
    prev_shape = shape


#show all the shapes in the path
#assign colors based on shape_id or randomly
shapes_in_path = sorted(set(dijkstra_path_shapes))
subset = transit_gdf[transit_gdf['shape_id'].isin(shapes_in_path)]
m = subset.explore(
    column='route_long_name',
    cmap='tab20',
    legend=True,
    tooltip=['route_long_name', 'route_short_name', 'trip_headsign'],
    style_kwds={'weight': 6, 'opacity': 0.9}
)
print(set(shapes_in_path))

stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)

path_gdf = stops_gdf[stops_gdf['stop_id'].isin(dijkstra_path_stops)]
m = path_gdf.explore(color='red', m=m)
openMap(m)